In [1]:
import pandas as pd
import numpy as np
import pathlib as Path
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [3]:
df_mls = pd.read_excel("Resources/FullMLS_PublicRecordData/Mult_List_Official_11_25_22.xlsx")
df_mls

,MLSNumber,Address,Category,Status,SoldPrice,OriginalPrice,CurrentPrice,Last List Price,Type,DOM,...,Parking,ExteriorFeatures,ExteriorMaterial,Main Roof,Foundation,PorchDeck,SwimmingPoolType,OtherStructures,OriginatingMLS,Structure Type
0,DENC518086,2615 Pecksniff Rd,RES,Closed,335000,330000,330000,NaN,NaN,5,...,NaN,"ExtensiveHardscape,Sidewalks,StoneRetainingWal...","BrickFront,VinylSiding",ArchitecturalShingle,NaN,"Patios,Porches",NaN,NaN,BRIGHT,Detached
1,DENC518982,4938 S Tupelo Turn,RES,Closed,200000,215500,210000,215500.0,NaN,47,...,NaN,NaN,"AluminumSiding,Brick,VinylSiding",Asphalt,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
2,DENC512992,15 Kristina Ct,RES,Closed,200000,200000,200000,NaN,NaN,15,...,NaN,NaN,VinylSiding,NaN,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
3,DENC512104,3251 Champions Dr,RES,Closed,200000,219900,214900,219900.0,NaN,11,...,NaN,NaN,Other,Shingle,NaN,Patios,NaN,NaN,BRIGHT,Interior Row/Townhouse
4,DENC503480,3706 Lafayette St,RES,Closed,200000,190000,190000,NaN,NaN,3,...,NaN,NaN,Asbestos,NaN,NaN,NaN,NaN,NaN,BRIGHT,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,DENC520114,101 Meriden Dr,RES,Closed,595000,595000,595000,NaN,NaN,5,...,AsphaltDriveway,"ExtensiveHardscape,StoneRetainingWalls","Stone,VinylSiding","Pitched,Shingle",NaN,"Brick,Patios,Porches",NaN,NaN,BRIGHT,Detached
5572,DENC520392,417 Snuff Mill Hill Rd,RES,Closed,595000,639000,639000,NaN,NaN,25,...,NaN,"ExtensiveHardscape,ExteriorLighting,FloodLight...","Cedar,Combination,Mixed,MixedPlumbing","Flat,Shake,Shingle",NaN,"Decks,Porches,Screened,WrapAround",NaN,NaN,BRIGHT,Detached
5573,DENC2014038,400 Wilson Ct,RES,Closed,600000,529900,529900,NaN,NaN,7,...,NaN,NaN,"AluminumSiding,Brick,VinylSiding",NaN,ConcretePerimeter,NaN,NaN,NaN,BRIGHT,Detached
5574,DENC2005484,918 Old Public Rd,RES,Closed,600000,620000,620000,635000.0,NaN,25,...,NaN,"Awnings,ExteriorLighting,LawnSprinkler",Brick,NaN,NaN,Decks,NaN,NaN,BRIGHT,Detached


In [4]:
df_mls.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 122 columns):
 #    Column                               Dtype         
---   ------                               -----         
 0    MLSNumber                            object        
 1    Address                              object        
 2    Category                             object        
 3    Status                               object        
 4    SoldPrice                            int64         
 5    OriginalPrice                        int64         
 6    CurrentPrice                         int64         
 7    Last List Price                      float64       
 8    Type                                 float64       
 9    DOM                                  int64         
 10   PMP                                  int64         
 11   ListDate                             datetime64[ns]
 12   StatusDate                           datetime64[ns]
 13   Agreement of Sal

In [5]:
DF_NEW = df_mls.drop(columns=["MLSNumber","Address","Category","Status", "Last List Price" , "Type"])


In [6]:
df_mls_new = DF_NEW.drop(DF_NEW.columns[[16,17,18,19]],axis=1)
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,ListDate,StatusDate,Agreement of Sale/Signed Lease Date,SettledDate,LastModifiedDate,...,Parking,ExteriorFeatures,ExteriorMaterial,Main Roof,Foundation,PorchDeck,SwimmingPoolType,OtherStructures,OriginatingMLS,Structure Type
0,335000,330000,330000,5,5,2020-12-11,2021-02-11,2020-12-16,2021-02-11,2022-08-23 05:13:22,...,NaN,"ExtensiveHardscape,Sidewalks,StoneRetainingWal...","BrickFront,VinylSiding",ArchitecturalShingle,NaN,"Patios,Porches",NaN,NaN,BRIGHT,Detached
1,200000,215500,210000,47,67,2021-01-07,2021-04-08,2021-02-21,2021-04-08,2022-08-23 09:36:35,...,NaN,NaN,"AluminumSiding,Brick,VinylSiding",Asphalt,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
2,200000,200000,200000,15,15,2020-12-28,2021-02-26,2021-01-09,2021-02-26,2022-08-22 23:22:50,...,NaN,NaN,VinylSiding,NaN,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
3,200000,219900,214900,11,11,2020-10-27,2020-12-14,2020-11-05,2020-12-11,2022-08-22 08:41:33,...,NaN,NaN,Other,Shingle,NaN,Patios,NaN,NaN,BRIGHT,Interior Row/Townhouse
4,200000,190000,190000,3,3,2020-06-20,2020-08-02,2020-06-21,2020-07-31,2022-08-21 05:33:33,...,NaN,NaN,Asbestos,NaN,NaN,NaN,NaN,NaN,BRIGHT,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,2021-06-24,2021-07-28,2021-06-28,2021-07-28,2022-08-25 01:18:10,...,AsphaltDriveway,"ExtensiveHardscape,StoneRetainingWalls","Stone,VinylSiding","Pitched,Shingle",NaN,"Brick,Patios,Porches",NaN,NaN,BRIGHT,Detached
5572,595000,639000,639000,25,25,2021-03-29,2021-06-10,2021-04-21,2021-06-10,2022-08-24 17:34:17,...,NaN,"ExtensiveHardscape,ExteriorLighting,FloodLight...","Cedar,Combination,Mixed,MixedPlumbing","Flat,Shake,Shingle",NaN,"Decks,Porches,Screened,WrapAround",NaN,NaN,BRIGHT,Detached
5573,600000,529900,529900,7,7,2022-01-14,2022-03-04,2022-01-19,2022-03-04,2022-09-05 03:16:06,...,NaN,NaN,"AluminumSiding,Brick,VinylSiding",NaN,ConcretePerimeter,NaN,NaN,NaN,BRIGHT,Detached
5574,600000,620000,620000,25,25,2021-08-26,2021-10-27,2021-09-18,2021-10-26,2022-09-13 18:12:32,...,NaN,"Awnings,ExteriorLighting,LawnSprinkler",Brick,NaN,NaN,Decks,NaN,NaN,BRIGHT,Detached


In [7]:
df_mls_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 112 columns):
 #    Column                               Dtype         
---   ------                               -----         
 0    SoldPrice                            int64         
 1    OriginalPrice                        int64         
 2    CurrentPrice                         int64         
 3    DOM                                  int64         
 4    PMP                                  int64         
 5    ListDate                             datetime64[ns]
 6    StatusDate                           datetime64[ns]
 7    Agreement of Sale/Signed Lease Date  datetime64[ns]
 8    SettledDate                          datetime64[ns]
 9    LastModifiedDate                     datetime64[ns]
 10   StreetNumber                         object        
 11   StreetName                           object        
 12   UnitNumber                           object        
 13   CrossStreet     

In [8]:
df_mls_new = df_mls_new.drop(df_mls_new.columns[10:18],axis=1)
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,ListDate,StatusDate,Agreement of Sale/Signed Lease Date,SettledDate,LastModifiedDate,...,Parking,ExteriorFeatures,ExteriorMaterial,Main Roof,Foundation,PorchDeck,SwimmingPoolType,OtherStructures,OriginatingMLS,Structure Type
0,335000,330000,330000,5,5,2020-12-11,2021-02-11,2020-12-16,2021-02-11,2022-08-23 05:13:22,...,NaN,"ExtensiveHardscape,Sidewalks,StoneRetainingWal...","BrickFront,VinylSiding",ArchitecturalShingle,NaN,"Patios,Porches",NaN,NaN,BRIGHT,Detached
1,200000,215500,210000,47,67,2021-01-07,2021-04-08,2021-02-21,2021-04-08,2022-08-23 09:36:35,...,NaN,NaN,"AluminumSiding,Brick,VinylSiding",Asphalt,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
2,200000,200000,200000,15,15,2020-12-28,2021-02-26,2021-01-09,2021-02-26,2022-08-22 23:22:50,...,NaN,NaN,VinylSiding,NaN,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
3,200000,219900,214900,11,11,2020-10-27,2020-12-14,2020-11-05,2020-12-11,2022-08-22 08:41:33,...,NaN,NaN,Other,Shingle,NaN,Patios,NaN,NaN,BRIGHT,Interior Row/Townhouse
4,200000,190000,190000,3,3,2020-06-20,2020-08-02,2020-06-21,2020-07-31,2022-08-21 05:33:33,...,NaN,NaN,Asbestos,NaN,NaN,NaN,NaN,NaN,BRIGHT,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,2021-06-24,2021-07-28,2021-06-28,2021-07-28,2022-08-25 01:18:10,...,AsphaltDriveway,"ExtensiveHardscape,StoneRetainingWalls","Stone,VinylSiding","Pitched,Shingle",NaN,"Brick,Patios,Porches",NaN,NaN,BRIGHT,Detached
5572,595000,639000,639000,25,25,2021-03-29,2021-06-10,2021-04-21,2021-06-10,2022-08-24 17:34:17,...,NaN,"ExtensiveHardscape,ExteriorLighting,FloodLight...","Cedar,Combination,Mixed,MixedPlumbing","Flat,Shake,Shingle",NaN,"Decks,Porches,Screened,WrapAround",NaN,NaN,BRIGHT,Detached
5573,600000,529900,529900,7,7,2022-01-14,2022-03-04,2022-01-19,2022-03-04,2022-09-05 03:16:06,...,NaN,NaN,"AluminumSiding,Brick,VinylSiding",NaN,ConcretePerimeter,NaN,NaN,NaN,BRIGHT,Detached
5574,600000,620000,620000,25,25,2021-08-26,2021-10-27,2021-09-18,2021-10-26,2022-09-13 18:12:32,...,NaN,"Awnings,ExteriorLighting,LawnSprinkler",Brick,NaN,NaN,Decks,NaN,NaN,BRIGHT,Detached


In [9]:
df_mls_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 104 columns):
 #    Column                               Dtype         
---   ------                               -----         
 0    SoldPrice                            int64         
 1    OriginalPrice                        int64         
 2    CurrentPrice                         int64         
 3    DOM                                  int64         
 4    PMP                                  int64         
 5    ListDate                             datetime64[ns]
 6    StatusDate                           datetime64[ns]
 7    Agreement of Sale/Signed Lease Date  datetime64[ns]
 8    SettledDate                          datetime64[ns]
 9    LastModifiedDate                     datetime64[ns]
 10   Subdivision                          object        
 11   School District                      object        
 12   Schools-Elementary                   object        
 13   Schools-Middle  

In [10]:
df_mls_new = df_mls_new.drop(df_mls_new.columns[9:20],axis=1)
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,ListDate,StatusDate,Agreement of Sale/Signed Lease Date,SettledDate,ListOfficePhone,...,Parking,ExteriorFeatures,ExteriorMaterial,Main Roof,Foundation,PorchDeck,SwimmingPoolType,OtherStructures,OriginatingMLS,Structure Type
0,335000,330000,330000,5,5,2020-12-11,2021-02-11,2020-12-16,2021-02-11,(302) 285-5100,...,NaN,"ExtensiveHardscape,Sidewalks,StoneRetainingWal...","BrickFront,VinylSiding",ArchitecturalShingle,NaN,"Patios,Porches",NaN,NaN,BRIGHT,Detached
1,200000,215500,210000,47,67,2021-01-07,2021-04-08,2021-02-21,2021-04-08,(302) 351-5000,...,NaN,NaN,"AluminumSiding,Brick,VinylSiding",Asphalt,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
2,200000,200000,200000,15,15,2020-12-28,2021-02-26,2021-01-09,2021-02-26,(302) 351-5000,...,NaN,NaN,VinylSiding,NaN,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
3,200000,219900,214900,11,11,2020-10-27,2020-12-14,2020-11-05,2020-12-11,(302) 834-3431,...,NaN,NaN,Other,Shingle,NaN,Patios,NaN,NaN,BRIGHT,Interior Row/Townhouse
4,200000,190000,190000,3,3,2020-06-20,2020-08-02,2020-06-21,2020-07-31,(302) 994-9600,...,NaN,NaN,Asbestos,NaN,NaN,NaN,NaN,NaN,BRIGHT,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,2021-06-24,2021-07-28,2021-06-28,2021-07-28,(610) 444-9090,...,AsphaltDriveway,"ExtensiveHardscape,StoneRetainingWalls","Stone,VinylSiding","Pitched,Shingle",NaN,"Brick,Patios,Porches",NaN,NaN,BRIGHT,Detached
5572,595000,639000,639000,25,25,2021-03-29,2021-06-10,2021-04-21,2021-06-10,(302) 571-8855,...,NaN,"ExtensiveHardscape,ExteriorLighting,FloodLight...","Cedar,Combination,Mixed,MixedPlumbing","Flat,Shake,Shingle",NaN,"Decks,Porches,Screened,WrapAround",NaN,NaN,BRIGHT,Detached
5573,600000,529900,529900,7,7,2022-01-14,2022-03-04,2022-01-19,2022-03-04,(302) 999-9999,...,NaN,NaN,"AluminumSiding,Brick,VinylSiding",NaN,ConcretePerimeter,NaN,NaN,NaN,BRIGHT,Detached
5574,600000,620000,620000,25,25,2021-08-26,2021-10-27,2021-09-18,2021-10-26,(888) 543-4829,...,NaN,"Awnings,ExteriorLighting,LawnSprinkler",Brick,NaN,NaN,Decks,NaN,NaN,BRIGHT,Detached


In [11]:
df_mls_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 93 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   SoldPrice                            5576 non-null   int64         
 1   OriginalPrice                        5576 non-null   int64         
 2   CurrentPrice                         5576 non-null   int64         
 3   DOM                                  5576 non-null   int64         
 4   PMP                                  5576 non-null   int64         
 5   ListDate                             5576 non-null   datetime64[ns]
 6   StatusDate                           5576 non-null   datetime64[ns]
 7   Agreement of Sale/Signed Lease Date  5576 non-null   datetime64[ns]
 8   SettledDate                          5576 non-null   datetime64[ns]
 9   ListOfficePhone                      5576 non-null   object        
 10  ListAgentNam

In [12]:
df_mls_new = df_mls_new.drop(df_mls_new.columns[9:30],axis=1)
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,ListDate,StatusDate,Agreement of Sale/Signed Lease Date,SettledDate,FinalBankOwned,...,Parking,ExteriorFeatures,ExteriorMaterial,Main Roof,Foundation,PorchDeck,SwimmingPoolType,OtherStructures,OriginatingMLS,Structure Type
0,335000,330000,330000,5,5,2020-12-11,2021-02-11,2020-12-16,2021-02-11,StandardSale,...,NaN,"ExtensiveHardscape,Sidewalks,StoneRetainingWal...","BrickFront,VinylSiding",ArchitecturalShingle,NaN,"Patios,Porches",NaN,NaN,BRIGHT,Detached
1,200000,215500,210000,47,67,2021-01-07,2021-04-08,2021-02-21,2021-04-08,StandardSale,...,NaN,NaN,"AluminumSiding,Brick,VinylSiding",Asphalt,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
2,200000,200000,200000,15,15,2020-12-28,2021-02-26,2021-01-09,2021-02-26,StandardSale,...,NaN,NaN,VinylSiding,NaN,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
3,200000,219900,214900,11,11,2020-10-27,2020-12-14,2020-11-05,2020-12-11,StandardSale,...,NaN,NaN,Other,Shingle,NaN,Patios,NaN,NaN,BRIGHT,Interior Row/Townhouse
4,200000,190000,190000,3,3,2020-06-20,2020-08-02,2020-06-21,2020-07-31,StandardSale,...,NaN,NaN,Asbestos,NaN,NaN,NaN,NaN,NaN,BRIGHT,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,2021-06-24,2021-07-28,2021-06-28,2021-07-28,StandardSale,...,AsphaltDriveway,"ExtensiveHardscape,StoneRetainingWalls","Stone,VinylSiding","Pitched,Shingle",NaN,"Brick,Patios,Porches",NaN,NaN,BRIGHT,Detached
5572,595000,639000,639000,25,25,2021-03-29,2021-06-10,2021-04-21,2021-06-10,StandardSale,...,NaN,"ExtensiveHardscape,ExteriorLighting,FloodLight...","Cedar,Combination,Mixed,MixedPlumbing","Flat,Shake,Shingle",NaN,"Decks,Porches,Screened,WrapAround",NaN,NaN,BRIGHT,Detached
5573,600000,529900,529900,7,7,2022-01-14,2022-03-04,2022-01-19,2022-03-04,StandardSale,...,NaN,NaN,"AluminumSiding,Brick,VinylSiding",NaN,ConcretePerimeter,NaN,NaN,NaN,BRIGHT,Detached
5574,600000,620000,620000,25,25,2021-08-26,2021-10-27,2021-09-18,2021-10-26,StandardSale,...,NaN,"Awnings,ExteriorLighting,LawnSprinkler",Brick,NaN,NaN,Decks,NaN,NaN,BRIGHT,Detached


In [13]:
df_mls_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 72 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   SoldPrice                            5576 non-null   int64         
 1   OriginalPrice                        5576 non-null   int64         
 2   CurrentPrice                         5576 non-null   int64         
 3   DOM                                  5576 non-null   int64         
 4   PMP                                  5576 non-null   int64         
 5   ListDate                             5576 non-null   datetime64[ns]
 6   StatusDate                           5576 non-null   datetime64[ns]
 7   Agreement of Sale/Signed Lease Date  5576 non-null   datetime64[ns]
 8   SettledDate                          5576 non-null   datetime64[ns]
 9   FinalBankOwned                       5575 non-null   object        
 10  Acres       

In [14]:
df_mls_new = df_mls_new.drop(df_mls_new.columns[11:22],axis=1)
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,ListDate,StatusDate,Agreement of Sale/Signed Lease Date,SettledDate,FinalBankOwned,...,Parking,ExteriorFeatures,ExteriorMaterial,Main Roof,Foundation,PorchDeck,SwimmingPoolType,OtherStructures,OriginatingMLS,Structure Type
0,335000,330000,330000,5,5,2020-12-11,2021-02-11,2020-12-16,2021-02-11,StandardSale,...,NaN,"ExtensiveHardscape,Sidewalks,StoneRetainingWal...","BrickFront,VinylSiding",ArchitecturalShingle,NaN,"Patios,Porches",NaN,NaN,BRIGHT,Detached
1,200000,215500,210000,47,67,2021-01-07,2021-04-08,2021-02-21,2021-04-08,StandardSale,...,NaN,NaN,"AluminumSiding,Brick,VinylSiding",Asphalt,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
2,200000,200000,200000,15,15,2020-12-28,2021-02-26,2021-01-09,2021-02-26,StandardSale,...,NaN,NaN,VinylSiding,NaN,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
3,200000,219900,214900,11,11,2020-10-27,2020-12-14,2020-11-05,2020-12-11,StandardSale,...,NaN,NaN,Other,Shingle,NaN,Patios,NaN,NaN,BRIGHT,Interior Row/Townhouse
4,200000,190000,190000,3,3,2020-06-20,2020-08-02,2020-06-21,2020-07-31,StandardSale,...,NaN,NaN,Asbestos,NaN,NaN,NaN,NaN,NaN,BRIGHT,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,2021-06-24,2021-07-28,2021-06-28,2021-07-28,StandardSale,...,AsphaltDriveway,"ExtensiveHardscape,StoneRetainingWalls","Stone,VinylSiding","Pitched,Shingle",NaN,"Brick,Patios,Porches",NaN,NaN,BRIGHT,Detached
5572,595000,639000,639000,25,25,2021-03-29,2021-06-10,2021-04-21,2021-06-10,StandardSale,...,NaN,"ExtensiveHardscape,ExteriorLighting,FloodLight...","Cedar,Combination,Mixed,MixedPlumbing","Flat,Shake,Shingle",NaN,"Decks,Porches,Screened,WrapAround",NaN,NaN,BRIGHT,Detached
5573,600000,529900,529900,7,7,2022-01-14,2022-03-04,2022-01-19,2022-03-04,StandardSale,...,NaN,NaN,"AluminumSiding,Brick,VinylSiding",NaN,ConcretePerimeter,NaN,NaN,NaN,BRIGHT,Detached
5574,600000,620000,620000,25,25,2021-08-26,2021-10-27,2021-09-18,2021-10-26,StandardSale,...,NaN,"Awnings,ExteriorLighting,LawnSprinkler",Brick,NaN,NaN,Decks,NaN,NaN,BRIGHT,Detached


In [15]:
df_mls_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 61 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   SoldPrice                            5576 non-null   int64         
 1   OriginalPrice                        5576 non-null   int64         
 2   CurrentPrice                         5576 non-null   int64         
 3   DOM                                  5576 non-null   int64         
 4   PMP                                  5576 non-null   int64         
 5   ListDate                             5576 non-null   datetime64[ns]
 6   StatusDate                           5576 non-null   datetime64[ns]
 7   Agreement of Sale/Signed Lease Date  5576 non-null   datetime64[ns]
 8   SettledDate                          5576 non-null   datetime64[ns]
 9   FinalBankOwned                       5575 non-null   object        
 10  Acres       

In [16]:
df_mls_new =  df_mls_new.dropna(axis='columns', how='all')
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,ListDate,StatusDate,Agreement of Sale/Signed Lease Date,SettledDate,FinalBankOwned,...,GarageFeatures,Parking,ExteriorFeatures,ExteriorMaterial,Main Roof,Foundation,PorchDeck,SwimmingPoolType,OriginatingMLS,Structure Type
0,335000,330000,330000,5,5,2020-12-11,2021-02-11,2020-12-16,2021-02-11,StandardSale,...,NaN,NaN,"ExtensiveHardscape,Sidewalks,StoneRetainingWal...","BrickFront,VinylSiding",ArchitecturalShingle,NaN,"Patios,Porches",NaN,BRIGHT,Detached
1,200000,215500,210000,47,67,2021-01-07,2021-04-08,2021-02-21,2021-04-08,StandardSale,...,NaN,NaN,NaN,"AluminumSiding,Brick,VinylSiding",Asphalt,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
2,200000,200000,200000,15,15,2020-12-28,2021-02-26,2021-01-09,2021-02-26,StandardSale,...,NaN,NaN,NaN,VinylSiding,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
3,200000,219900,214900,11,11,2020-10-27,2020-12-14,2020-11-05,2020-12-11,StandardSale,...,NaN,NaN,NaN,Other,Shingle,NaN,Patios,NaN,BRIGHT,Interior Row/Townhouse
4,200000,190000,190000,3,3,2020-06-20,2020-08-02,2020-06-21,2020-07-31,StandardSale,...,NaN,NaN,NaN,Asbestos,NaN,NaN,NaN,NaN,BRIGHT,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,2021-06-24,2021-07-28,2021-06-28,2021-07-28,StandardSale,...,AsphaltDriveway,AsphaltDriveway,"ExtensiveHardscape,StoneRetainingWalls","Stone,VinylSiding","Pitched,Shingle",NaN,"Brick,Patios,Porches",NaN,BRIGHT,Detached
5572,595000,639000,639000,25,25,2021-03-29,2021-06-10,2021-04-21,2021-06-10,StandardSale,...,NaN,NaN,"ExtensiveHardscape,ExteriorLighting,FloodLight...","Cedar,Combination,Mixed,MixedPlumbing","Flat,Shake,Shingle",NaN,"Decks,Porches,Screened,WrapAround",NaN,BRIGHT,Detached
5573,600000,529900,529900,7,7,2022-01-14,2022-03-04,2022-01-19,2022-03-04,StandardSale,...,NaN,NaN,NaN,"AluminumSiding,Brick,VinylSiding",NaN,ConcretePerimeter,NaN,NaN,BRIGHT,Detached
5574,600000,620000,620000,25,25,2021-08-26,2021-10-27,2021-09-18,2021-10-26,StandardSale,...,NaN,NaN,"Awnings,ExteriorLighting,LawnSprinkler",Brick,NaN,NaN,Decks,NaN,BRIGHT,Detached


In [17]:
df_mls_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 58 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   SoldPrice                            5576 non-null   int64         
 1   OriginalPrice                        5576 non-null   int64         
 2   CurrentPrice                         5576 non-null   int64         
 3   DOM                                  5576 non-null   int64         
 4   PMP                                  5576 non-null   int64         
 5   ListDate                             5576 non-null   datetime64[ns]
 6   StatusDate                           5576 non-null   datetime64[ns]
 7   Agreement of Sale/Signed Lease Date  5576 non-null   datetime64[ns]
 8   SettledDate                          5576 non-null   datetime64[ns]
 9   FinalBankOwned                       5575 non-null   object        
 10  Acres       

In [18]:
df_mls_new = df_mls_new.drop(df_mls_new.columns[5:9],axis=1)
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,FinalBankOwned,Acres,Age,InteriorSqFt,Interior SqFt Source,...,GarageFeatures,Parking,ExteriorFeatures,ExteriorMaterial,Main Roof,Foundation,PorchDeck,SwimmingPoolType,OriginatingMLS,Structure Type
0,335000,330000,330000,5,5,StandardSale,0.26,1958,2099,Estimated,...,NaN,NaN,"ExtensiveHardscape,Sidewalks,StoneRetainingWal...","BrickFront,VinylSiding",ArchitecturalShingle,NaN,"Patios,Porches",NaN,BRIGHT,Detached
1,200000,215500,210000,47,67,StandardSale,0.06,1976,1927,Assessor,...,NaN,NaN,NaN,"AluminumSiding,Brick,VinylSiding",Asphalt,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
2,200000,200000,200000,15,15,StandardSale,0.05,1985,1150,Assessor,...,NaN,NaN,NaN,VinylSiding,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
3,200000,219900,214900,11,11,StandardSale,0.05,1985,1425,Assessor,...,NaN,NaN,NaN,Other,Shingle,NaN,Patios,NaN,BRIGHT,Interior Row/Townhouse
4,200000,190000,190000,3,3,StandardSale,0.17,1957,925,Assessor,...,NaN,NaN,NaN,Asbestos,NaN,NaN,NaN,NaN,BRIGHT,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,StandardSale,1.03,1958,3045,Estimated,...,AsphaltDriveway,AsphaltDriveway,"ExtensiveHardscape,StoneRetainingWalls","Stone,VinylSiding","Pitched,Shingle",NaN,"Brick,Patios,Porches",NaN,BRIGHT,Detached
5572,595000,639000,639000,25,25,StandardSale,0.75,1973,3811,Assessor,...,NaN,NaN,"ExtensiveHardscape,ExteriorLighting,FloodLight...","Cedar,Combination,Mixed,MixedPlumbing","Flat,Shake,Shingle",NaN,"Decks,Porches,Screened,WrapAround",NaN,BRIGHT,Detached
5573,600000,529900,529900,7,7,StandardSale,0.39,1989,3191,Assessor,...,NaN,NaN,NaN,"AluminumSiding,Brick,VinylSiding",NaN,ConcretePerimeter,NaN,NaN,BRIGHT,Detached
5574,600000,620000,620000,25,25,StandardSale,0.54,1992,3425,Assessor,...,NaN,NaN,"Awnings,ExteriorLighting,LawnSprinkler",Brick,NaN,NaN,Decks,NaN,BRIGHT,Detached


In [19]:
df_mls_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 54 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SoldPrice                  5576 non-null   int64  
 1   OriginalPrice              5576 non-null   int64  
 2   CurrentPrice               5576 non-null   int64  
 3   DOM                        5576 non-null   int64  
 4   PMP                        5576 non-null   int64  
 5   FinalBankOwned             5575 non-null   object 
 6   Acres                      5354 non-null   float64
 7   Age                        5576 non-null   int64  
 8   InteriorSqFt               5576 non-null   int64  
 9   Interior SqFt Source       5576 non-null   object 
 10  AboveGradeSqFt             5576 non-null   int64  
 11  BelowGradeSqFt             5394 non-null   float64
 12  PropertyCondition          1916 non-null   object 
 13  Bedrooms                   5571 non-null   float

In [20]:
df_mls_new = df_mls_new.drop(df_mls_new.columns[17:19],axis=1)
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,FinalBankOwned,Acres,Age,InteriorSqFt,Interior SqFt Source,...,GarageFeatures,Parking,ExteriorFeatures,ExteriorMaterial,Main Roof,Foundation,PorchDeck,SwimmingPoolType,OriginatingMLS,Structure Type
0,335000,330000,330000,5,5,StandardSale,0.26,1958,2099,Estimated,...,NaN,NaN,"ExtensiveHardscape,Sidewalks,StoneRetainingWal...","BrickFront,VinylSiding",ArchitecturalShingle,NaN,"Patios,Porches",NaN,BRIGHT,Detached
1,200000,215500,210000,47,67,StandardSale,0.06,1976,1927,Assessor,...,NaN,NaN,NaN,"AluminumSiding,Brick,VinylSiding",Asphalt,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
2,200000,200000,200000,15,15,StandardSale,0.05,1985,1150,Assessor,...,NaN,NaN,NaN,VinylSiding,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
3,200000,219900,214900,11,11,StandardSale,0.05,1985,1425,Assessor,...,NaN,NaN,NaN,Other,Shingle,NaN,Patios,NaN,BRIGHT,Interior Row/Townhouse
4,200000,190000,190000,3,3,StandardSale,0.17,1957,925,Assessor,...,NaN,NaN,NaN,Asbestos,NaN,NaN,NaN,NaN,BRIGHT,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,StandardSale,1.03,1958,3045,Estimated,...,AsphaltDriveway,AsphaltDriveway,"ExtensiveHardscape,StoneRetainingWalls","Stone,VinylSiding","Pitched,Shingle",NaN,"Brick,Patios,Porches",NaN,BRIGHT,Detached
5572,595000,639000,639000,25,25,StandardSale,0.75,1973,3811,Assessor,...,NaN,NaN,"ExtensiveHardscape,ExteriorLighting,FloodLight...","Cedar,Combination,Mixed,MixedPlumbing","Flat,Shake,Shingle",NaN,"Decks,Porches,Screened,WrapAround",NaN,BRIGHT,Detached
5573,600000,529900,529900,7,7,StandardSale,0.39,1989,3191,Assessor,...,NaN,NaN,NaN,"AluminumSiding,Brick,VinylSiding",NaN,ConcretePerimeter,NaN,NaN,BRIGHT,Detached
5574,600000,620000,620000,25,25,StandardSale,0.54,1992,3425,Assessor,...,NaN,NaN,"Awnings,ExteriorLighting,LawnSprinkler",Brick,NaN,NaN,Decks,NaN,BRIGHT,Detached


In [21]:
df_mls_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 52 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SoldPrice                  5576 non-null   int64  
 1   OriginalPrice              5576 non-null   int64  
 2   CurrentPrice               5576 non-null   int64  
 3   DOM                        5576 non-null   int64  
 4   PMP                        5576 non-null   int64  
 5   FinalBankOwned             5575 non-null   object 
 6   Acres                      5354 non-null   float64
 7   Age                        5576 non-null   int64  
 8   InteriorSqFt               5576 non-null   int64  
 9   Interior SqFt Source       5576 non-null   object 
 10  AboveGradeSqFt             5576 non-null   int64  
 11  BelowGradeSqFt             5394 non-null   float64
 12  PropertyCondition          1916 non-null   object 
 13  Bedrooms                   5571 non-null   float

In [22]:
df_mls_new = df_mls_new.drop(df_mls_new.columns[19:29],axis=1)
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,FinalBankOwned,Acres,Age,InteriorSqFt,Interior SqFt Source,...,GarageFeatures,Parking,ExteriorFeatures,ExteriorMaterial,Main Roof,Foundation,PorchDeck,SwimmingPoolType,OriginatingMLS,Structure Type
0,335000,330000,330000,5,5,StandardSale,0.26,1958,2099,Estimated,...,NaN,NaN,"ExtensiveHardscape,Sidewalks,StoneRetainingWal...","BrickFront,VinylSiding",ArchitecturalShingle,NaN,"Patios,Porches",NaN,BRIGHT,Detached
1,200000,215500,210000,47,67,StandardSale,0.06,1976,1927,Assessor,...,NaN,NaN,NaN,"AluminumSiding,Brick,VinylSiding",Asphalt,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
2,200000,200000,200000,15,15,StandardSale,0.05,1985,1150,Assessor,...,NaN,NaN,NaN,VinylSiding,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
3,200000,219900,214900,11,11,StandardSale,0.05,1985,1425,Assessor,...,NaN,NaN,NaN,Other,Shingle,NaN,Patios,NaN,BRIGHT,Interior Row/Townhouse
4,200000,190000,190000,3,3,StandardSale,0.17,1957,925,Assessor,...,NaN,NaN,NaN,Asbestos,NaN,NaN,NaN,NaN,BRIGHT,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,StandardSale,1.03,1958,3045,Estimated,...,AsphaltDriveway,AsphaltDriveway,"ExtensiveHardscape,StoneRetainingWalls","Stone,VinylSiding","Pitched,Shingle",NaN,"Brick,Patios,Porches",NaN,BRIGHT,Detached
5572,595000,639000,639000,25,25,StandardSale,0.75,1973,3811,Assessor,...,NaN,NaN,"ExtensiveHardscape,ExteriorLighting,FloodLight...","Cedar,Combination,Mixed,MixedPlumbing","Flat,Shake,Shingle",NaN,"Decks,Porches,Screened,WrapAround",NaN,BRIGHT,Detached
5573,600000,529900,529900,7,7,StandardSale,0.39,1989,3191,Assessor,...,NaN,NaN,NaN,"AluminumSiding,Brick,VinylSiding",NaN,ConcretePerimeter,NaN,NaN,BRIGHT,Detached
5574,600000,620000,620000,25,25,StandardSale,0.54,1992,3425,Assessor,...,NaN,NaN,"Awnings,ExteriorLighting,LawnSprinkler",Brick,NaN,NaN,Decks,NaN,BRIGHT,Detached


In [23]:
df_mls_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SoldPrice                  5576 non-null   int64  
 1   OriginalPrice              5576 non-null   int64  
 2   CurrentPrice               5576 non-null   int64  
 3   DOM                        5576 non-null   int64  
 4   PMP                        5576 non-null   int64  
 5   FinalBankOwned             5575 non-null   object 
 6   Acres                      5354 non-null   float64
 7   Age                        5576 non-null   int64  
 8   InteriorSqFt               5576 non-null   int64  
 9   Interior SqFt Source       5576 non-null   object 
 10  AboveGradeSqFt             5576 non-null   int64  
 11  BelowGradeSqFt             5394 non-null   float64
 12  PropertyCondition          1916 non-null   object 
 13  Bedrooms                   5571 non-null   float

In [24]:
df_mls_new = df_mls_new.drop(df_mls_new.columns[21:28],axis=1)
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,FinalBankOwned,Acres,Age,InteriorSqFt,Interior SqFt Source,...,GarageFeatures,Parking,ExteriorFeatures,ExteriorMaterial,Main Roof,Foundation,PorchDeck,SwimmingPoolType,OriginatingMLS,Structure Type
0,335000,330000,330000,5,5,StandardSale,0.26,1958,2099,Estimated,...,NaN,NaN,"ExtensiveHardscape,Sidewalks,StoneRetainingWal...","BrickFront,VinylSiding",ArchitecturalShingle,NaN,"Patios,Porches",NaN,BRIGHT,Detached
1,200000,215500,210000,47,67,StandardSale,0.06,1976,1927,Assessor,...,NaN,NaN,NaN,"AluminumSiding,Brick,VinylSiding",Asphalt,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
2,200000,200000,200000,15,15,StandardSale,0.05,1985,1150,Assessor,...,NaN,NaN,NaN,VinylSiding,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
3,200000,219900,214900,11,11,StandardSale,0.05,1985,1425,Assessor,...,NaN,NaN,NaN,Other,Shingle,NaN,Patios,NaN,BRIGHT,Interior Row/Townhouse
4,200000,190000,190000,3,3,StandardSale,0.17,1957,925,Assessor,...,NaN,NaN,NaN,Asbestos,NaN,NaN,NaN,NaN,BRIGHT,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,StandardSale,1.03,1958,3045,Estimated,...,AsphaltDriveway,AsphaltDriveway,"ExtensiveHardscape,StoneRetainingWalls","Stone,VinylSiding","Pitched,Shingle",NaN,"Brick,Patios,Porches",NaN,BRIGHT,Detached
5572,595000,639000,639000,25,25,StandardSale,0.75,1973,3811,Assessor,...,NaN,NaN,"ExtensiveHardscape,ExteriorLighting,FloodLight...","Cedar,Combination,Mixed,MixedPlumbing","Flat,Shake,Shingle",NaN,"Decks,Porches,Screened,WrapAround",NaN,BRIGHT,Detached
5573,600000,529900,529900,7,7,StandardSale,0.39,1989,3191,Assessor,...,NaN,NaN,NaN,"AluminumSiding,Brick,VinylSiding",NaN,ConcretePerimeter,NaN,NaN,BRIGHT,Detached
5574,600000,620000,620000,25,25,StandardSale,0.54,1992,3425,Assessor,...,NaN,NaN,"Awnings,ExteriorLighting,LawnSprinkler",Brick,NaN,NaN,Decks,NaN,BRIGHT,Detached


In [25]:
df_mls_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   SoldPrice             5576 non-null   int64  
 1   OriginalPrice         5576 non-null   int64  
 2   CurrentPrice          5576 non-null   int64  
 3   DOM                   5576 non-null   int64  
 4   PMP                   5576 non-null   int64  
 5   FinalBankOwned        5575 non-null   object 
 6   Acres                 5354 non-null   float64
 7   Age                   5576 non-null   int64  
 8   InteriorSqFt          5576 non-null   int64  
 9   Interior SqFt Source  5576 non-null   object 
 10  AboveGradeSqFt        5576 non-null   int64  
 11  BelowGradeSqFt        5394 non-null   float64
 12  PropertyCondition     1916 non-null   object 
 13  Bedrooms              5571 non-null   float64
 14  Baths                 5571 non-null   float64
 15  BathsFull            

In [26]:
df_mls_new = df_mls_new.drop(df_mls_new.columns[20:24],axis=1)
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,FinalBankOwned,Acres,Age,InteriorSqFt,Interior SqFt Source,...,GarageFeatures,Parking,ExteriorFeatures,ExteriorMaterial,Main Roof,Foundation,PorchDeck,SwimmingPoolType,OriginatingMLS,Structure Type
0,335000,330000,330000,5,5,StandardSale,0.26,1958,2099,Estimated,...,NaN,NaN,"ExtensiveHardscape,Sidewalks,StoneRetainingWal...","BrickFront,VinylSiding",ArchitecturalShingle,NaN,"Patios,Porches",NaN,BRIGHT,Detached
1,200000,215500,210000,47,67,StandardSale,0.06,1976,1927,Assessor,...,NaN,NaN,NaN,"AluminumSiding,Brick,VinylSiding",Asphalt,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
2,200000,200000,200000,15,15,StandardSale,0.05,1985,1150,Assessor,...,NaN,NaN,NaN,VinylSiding,NaN,NaN,NaN,NaN,BRIGHT,Interior Row/Townhouse
3,200000,219900,214900,11,11,StandardSale,0.05,1985,1425,Assessor,...,NaN,NaN,NaN,Other,Shingle,NaN,Patios,NaN,BRIGHT,Interior Row/Townhouse
4,200000,190000,190000,3,3,StandardSale,0.17,1957,925,Assessor,...,NaN,NaN,NaN,Asbestos,NaN,NaN,NaN,NaN,BRIGHT,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,StandardSale,1.03,1958,3045,Estimated,...,AsphaltDriveway,AsphaltDriveway,"ExtensiveHardscape,StoneRetainingWalls","Stone,VinylSiding","Pitched,Shingle",NaN,"Brick,Patios,Porches",NaN,BRIGHT,Detached
5572,595000,639000,639000,25,25,StandardSale,0.75,1973,3811,Assessor,...,NaN,NaN,"ExtensiveHardscape,ExteriorLighting,FloodLight...","Cedar,Combination,Mixed,MixedPlumbing","Flat,Shake,Shingle",NaN,"Decks,Porches,Screened,WrapAround",NaN,BRIGHT,Detached
5573,600000,529900,529900,7,7,StandardSale,0.39,1989,3191,Assessor,...,NaN,NaN,NaN,"AluminumSiding,Brick,VinylSiding",NaN,ConcretePerimeter,NaN,NaN,BRIGHT,Detached
5574,600000,620000,620000,25,25,StandardSale,0.54,1992,3425,Assessor,...,NaN,NaN,"Awnings,ExteriorLighting,LawnSprinkler",Brick,NaN,NaN,Decks,NaN,BRIGHT,Detached


In [27]:
df_mls_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   SoldPrice             5576 non-null   int64  
 1   OriginalPrice         5576 non-null   int64  
 2   CurrentPrice          5576 non-null   int64  
 3   DOM                   5576 non-null   int64  
 4   PMP                   5576 non-null   int64  
 5   FinalBankOwned        5575 non-null   object 
 6   Acres                 5354 non-null   float64
 7   Age                   5576 non-null   int64  
 8   InteriorSqFt          5576 non-null   int64  
 9   Interior SqFt Source  5576 non-null   object 
 10  AboveGradeSqFt        5576 non-null   int64  
 11  BelowGradeSqFt        5394 non-null   float64
 12  PropertyCondition     1916 non-null   object 
 13  Bedrooms              5571 non-null   float64
 14  Baths                 5571 non-null   float64
 15  BathsFull            

In [28]:
df_mls_new = df_mls_new.drop(df_mls_new.columns[21:31],axis=1)
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,FinalBankOwned,Acres,Age,InteriorSqFt,Interior SqFt Source,...,BelowGradeSqFt,PropertyCondition,Bedrooms,Baths,BathsFull,PartialBaths,NumberofStories,RoomCount,Fireplace YN,GarageSpaces
0,335000,330000,330000,5,5,StandardSale,0.26,1958,2099,Estimated,...,0.0,NaN,4.0,2.0,1.0,1.0,"Main,Upper1",8.0,NaN,1.0
1,200000,215500,210000,47,67,StandardSale,0.06,1976,1927,Assessor,...,352.0,Average,3.0,2.0,1.0,1.0,"Main,Upper1",NaN,Yes,NaN
2,200000,200000,200000,15,15,StandardSale,0.05,1985,1150,Assessor,...,0.0,NaN,2.0,2.0,1.0,1.0,"Main,Upper1",NaN,NaN,NaN
3,200000,219900,214900,11,11,StandardSale,0.05,1985,1425,Assessor,...,0.0,Good,2.0,3.0,2.0,1.0,"Main,Upper1",NaN,Yes,1.0
4,200000,190000,190000,3,3,StandardSale,0.17,1957,925,Assessor,...,0.0,VeryGood,3.0,2.0,1.0,1.0,Main,NaN,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,StandardSale,1.03,1958,3045,Estimated,...,0.0,VeryGood,3.0,2.0,2.0,NaN,"Lower1,Main",NaN,Yes,2.0
5572,595000,639000,639000,25,25,StandardSale,0.75,1973,3811,Assessor,...,636.0,"Excellent,VeryGood",4.0,4.0,3.0,1.0,"Lower1,Main,Upper1",NaN,Yes,2.0
5573,600000,529900,529900,7,7,StandardSale,0.39,1989,3191,Assessor,...,341.0,NaN,4.0,3.0,2.0,1.0,"Lower1,Main,Upper1",NaN,NaN,2.0
5574,600000,620000,620000,25,25,StandardSale,0.54,1992,3425,Assessor,...,0.0,"Excellent,VeryGood,Good",4.0,3.0,2.0,1.0,"Lower1,Main,Upper1",9.0,Yes,2.0


In [29]:
df_mls_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   SoldPrice             5576 non-null   int64  
 1   OriginalPrice         5576 non-null   int64  
 2   CurrentPrice          5576 non-null   int64  
 3   DOM                   5576 non-null   int64  
 4   PMP                   5576 non-null   int64  
 5   FinalBankOwned        5575 non-null   object 
 6   Acres                 5354 non-null   float64
 7   Age                   5576 non-null   int64  
 8   InteriorSqFt          5576 non-null   int64  
 9   Interior SqFt Source  5576 non-null   object 
 10  AboveGradeSqFt        5576 non-null   int64  
 11  BelowGradeSqFt        5394 non-null   float64
 12  PropertyCondition     1916 non-null   object 
 13  Bedrooms              5571 non-null   float64
 14  Baths                 5571 non-null   float64
 15  BathsFull            

In [30]:
df_mls_new = df_mls_new.drop(df_mls_new.columns[[5,9,12,17,19]],axis=1)
df_mls_new

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,Acres,Age,InteriorSqFt,AboveGradeSqFt,BelowGradeSqFt,Bedrooms,Baths,BathsFull,PartialBaths,RoomCount,GarageSpaces
0,335000,330000,330000,5,5,0.26,1958,2099,2099,0.0,4.0,2.0,1.0,1.0,8.0,1.0
1,200000,215500,210000,47,67,0.06,1976,1927,1575,352.0,3.0,2.0,1.0,1.0,NaN,NaN
2,200000,200000,200000,15,15,0.05,1985,1150,1150,0.0,2.0,2.0,1.0,1.0,NaN,NaN
3,200000,219900,214900,11,11,0.05,1985,1425,1425,0.0,2.0,3.0,2.0,1.0,NaN,1.0
4,200000,190000,190000,3,3,0.17,1957,925,925,0.0,3.0,2.0,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,595000,595000,595000,5,5,1.03,1958,3045,3045,0.0,3.0,2.0,2.0,NaN,NaN,2.0
5572,595000,639000,639000,25,25,0.75,1973,3811,3175,636.0,4.0,4.0,3.0,1.0,NaN,2.0
5573,600000,529900,529900,7,7,0.39,1989,3191,2850,341.0,4.0,3.0,2.0,1.0,NaN,2.0
5574,600000,620000,620000,25,25,0.54,1992,3425,3425,0.0,4.0,3.0,2.0,1.0,9.0,2.0


In [31]:
df_mls_new.isna().sum()

SoldPrice            0
OriginalPrice        0
CurrentPrice         0
DOM                  0
PMP                  0
Acres              222
Age                  0
InteriorSqFt         0
AboveGradeSqFt       0
BelowGradeSqFt     182
Bedrooms             5
Baths                5
BathsFull            5
PartialBaths      1637
RoomCount         4570
GarageSpaces      2170
dtype: int64

In [32]:
df_mls_new[["Acres","BelowGradeSqFt","Bedrooms","Baths","BathsFull","PartialBaths","GarageSpaces"]] =  df_mls_new[["Acres","BelowGradeSqFt","Bedrooms","Baths","BathsFull","PartialBaths","GarageSpaces"]].fillna(value=df_mls_new[["Acres","BelowGradeSqFt","Bedrooms","Baths","BathsFull","PartialBaths","GarageSpaces"]].mean())

In [33]:
df_mls_new = df_mls_new.drop(columns="RoomCount")

In [34]:
df_mls_new.isna().sum()

SoldPrice         0
OriginalPrice     0
CurrentPrice      0
DOM               0
PMP               0
Acres             0
Age               0
InteriorSqFt      0
AboveGradeSqFt    0
BelowGradeSqFt    0
Bedrooms          0
Baths             0
BathsFull         0
PartialBaths      0
GarageSpaces      0
dtype: int64

In [35]:
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [36]:
mls_scaled = data_scaler.fit_transform(df_mls_new)
mls_scaled

array([[ 0.11423801,  0.06413778,  0.08181996, ..., -1.21385838,
         0.14519421, -0.12737198],
       [-1.25592515, -0.94826515, -1.12377706, ..., -1.21385838,
         0.14519421,  0.        ],
       [-1.25592515, -1.08531532, -1.22424348, ..., -1.21385838,
         0.14519421,  0.        ],
       ...,
       [ 2.80381756,  1.83164297,  2.09014367, ...,  0.33893267,
         0.14519421,  0.00603176],
       [ 2.80381756,  2.62830239,  2.9953461 , ...,  0.33893267,
         0.14519421,  0.00603176],
       [ 0.26647836,  0.24097672,  0.2827528 , ...,  0.33893267,
        -2.91320149, -0.12737198]])

In [37]:
df2 = pd.get_dummies(df_mls_new)
X = df_mls_new.drop(columns=["SoldPrice"])


#Create our target
y = df_mls_new["SoldPrice"]

In [38]:

X.describe()


,OriginalPrice,CurrentPrice,DOM,PMP,Acres,Age,InteriorSqFt,AboveGradeSqFt,BelowGradeSqFt,Bedrooms,Baths,BathsFull,PartialBaths,GarageSpaces
count,5.576000e+03,5576.000000,5576.000000,5576.000000,5576.000000,5576.000000,5576.000000,5576.000000,5576.000000,5576.000000,5576.000000,5576.000000,5576.000000,5576.000000
mean,3.227462e+05,321855.988881,17.510581,19.248924,1.722159,1958.652260,2005.095768,1808.543938,203.183723,3.317537,2.455215,1.781727,0.952526,1.954786
std,1.131074e+05,99544.673267,43.678523,50.091712,88.491492,94.847263,880.013932,734.820820,348.632576,0.774397,0.818133,0.644059,0.326998,7.496714
min,1.399000e+05,174900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.407485e+05,244975.000000,4.000000,4.000000,0.100000,1950.000000,1450.000000,1375.000000,0.000000,3.000000,2.000000,1.000000,0.952526,1.000000
50%,2.950000e+05,295000.000000,6.000000,6.000000,0.180000,1962.000000,1925.000000,1725.000000,0.000000,3.000000,2.000000,2.000000,1.000000,1.954786
75%,3.750000e+05,375000.000000,15.000000,15.000000,0.290000,1986.000000,2475.250000,2179.750000,350.000000,4.000000,3.000000,2.000000,1.000000,2.000000
max,3.899000e+06,699000.000000,1682.000000,1682.000000,6534.000000,2021.000000,8607.000000,6425.000000,2585.000000,8.000000,6.000000,6.000000,3.000000,229.000000


In [39]:
len(y)

5576

In [40]:
from sklearn.linear_model import LinearRegression

In [41]:
model = LinearRegression()

In [42]:
model.fit(X,y)

LinearRegression()

In [43]:
y_pred = model.predict(X)
print(y_pred.shape)

(5576,)


In [44]:
y_pred

array([333959.55652449, 211833.07545602, 203929.18963031, ...,
       529504.72730186, 615730.09760634, 347504.30542313])

In [45]:
model.score(X,y)

0.979189206927665

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
X_train.shape



(4460, 14)

In [47]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [48]:
from sklearn.linear_model import ElasticNet
eNet = ElasticNet(random_state=0)
eNet.fit(X_train_scaled,y_train)

pred = eNet.predict(X_test_scaled)
pred

array([265573.63156289, 319651.25288545, 310103.84304249, ...,
       288038.9407642 , 294666.78455693, 300076.86681008])

In [49]:
eNet.score(X_test_scaled,y_test)

0.8237220961626469

In [50]:
from sklearn.model_selection import cross_val_score

cv_acc = cross_val_score(eNet,X_test_scaled,y_test, cv=5,scoring=None)
cv_acc

array([0.89559229, 0.27861294, 0.90580341, 0.89129907, 0.88845859])

In [51]:
from sklearn.model_selection import cross_validate
cv_score = cross_validate(eNet, X_test_scaled,y_test,cv=3)
sorted(cv_score.keys())
cv_score

{'fit_time': array([0.00125813, 0.00068784, 0.00062275]),
 'score_time': array([0.00021386, 0.0001862 , 0.00018835]),
 'test_score': array([0.56680436, 0.90446032, 0.88150807])}

In [52]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

In [53]:
print(r2_score(y_test,pred))

0.8237220961626469


In [54]:
print(mse(y_test,pred)**0.5)

41372.849992878146


In [55]:
df_mls_new.head(1)

,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,Acres,Age,InteriorSqFt,AboveGradeSqFt,BelowGradeSqFt,Bedrooms,Baths,BathsFull,PartialBaths,GarageSpaces
0,335000,330000,330000,5,5,0.26,1958,2099,2099,0.0,4.0,2.0,1.0,1.0,1.0


In [56]:
df_mls_new.columns

Index(['SoldPrice', 'OriginalPrice', 'CurrentPrice', 'DOM', 'PMP', 'Acres',
       'Age', 'InteriorSqFt', 'AboveGradeSqFt', 'BelowGradeSqFt', 'Bedrooms',
       'Baths', 'BathsFull', 'PartialBaths', 'GarageSpaces'],
      dtype='object')

In [57]:
d = {"SoldPrice":[0],"OriginalPrice" :[300000],"CurrentPrice":[340000],"DOM":[7],"PMP":[9],"Acres":[0.3],"Age":[1980],"InteriorSqFt":[1900],"AboveGradeSqFt":[2099],"BelowGradeSqFt":[0.0],"Bedrooms":[6.0],"Baths":[4.0],"BathsFull":[2.0],"PartialBaths":[2.0],"GarageSpaces":[1.0]  }
testing_df = pd.DataFrame(data=d,)
testing_df


,SoldPrice,OriginalPrice,CurrentPrice,DOM,PMP,Acres,Age,InteriorSqFt,AboveGradeSqFt,BelowGradeSqFt,Bedrooms,Baths,BathsFull,PartialBaths,GarageSpaces
0,0,300000,340000,7,9,0.3,1980,1900,2099,0.0,6.0,4.0,2.0,2.0,1.0


In [58]:
testing_df = testing_df.drop(columns="SoldPrice")

In [59]:
TEST = testing_df

In [60]:
linpred = model.predict(TEST)

In [61]:
linpred

array([345186.30279985])

In [62]:
e_net_scaler = scaler.fit(TEST)
e_scaled = e_net_scaler.transform(TEST)

In [63]:
e_pred = eNet.predict(e_scaled)
e_pred

array([324093.54058296])

In [64]:
testing_df["PredictedBasicLinearRegressionPrice"] = linpred
testing_df["PredictedElasticNetPrice"] = e_pred

In [65]:
testing_df

,OriginalPrice,CurrentPrice,DOM,PMP,Acres,Age,InteriorSqFt,AboveGradeSqFt,BelowGradeSqFt,Bedrooms,Baths,BathsFull,PartialBaths,GarageSpaces,PredictedBasicLinearRegressionPrice,PredictedElasticNetPrice
0,300000,340000,7,9,0.3,1980,1900,2099,0.0,6.0,4.0,2.0,2.0,1.0,345186.3028,324093.540583


In [66]:
testing_df=testing_df.reindex(columns=["OriginalPrice","CurrentPrice","PredictedBasicLinearRegressionPrice","PredictedElasticNetPrice","DOM","PMP","Acres","Age","InteriorSqFt","AboveGradeSqFt","BelowGradeSqFt","Bedrooms","Baths","BathsFull","PartialBaths","GarageSpaces"])

In [67]:
testing_df

,OriginalPrice,CurrentPrice,PredictedBasicLinearRegressionPrice,PredictedElasticNetPrice,DOM,PMP,Acres,Age,InteriorSqFt,AboveGradeSqFt,BelowGradeSqFt,Bedrooms,Baths,BathsFull,PartialBaths,GarageSpaces
0,300000,340000,345186.3028,324093.540583,7,9,0.3,1980,1900,2099,0.0,6.0,4.0,2.0,2.0,1.0
